# Stochastic gradient descent with momentum from scratch

## Theory
Stochastic Gradient Descent with Momentum is built on Stochastic Gradient Descent. It is fed with a random vector from known data in the training dataset.<br> All the work on calculating the gradient is happening one line at a time. Then we move on to the next random line.<br> It takes a long time, so to speed everything up, we use momentum

Very interesting, but I found 2 kinds of formulas in different articles: <br>
1. [An overview of gradient descent optimization algorithms](https://arxiv.org/pdf/1609.04747.pdf): <br>
$ v_t = \gamma * v_ {t-1} + \alpha \nabla_ \theta f (\theta) $ <br>
$ \theta = \theta - v_t $
2. [An Improved Analysis of Stochastic Gradient Descentwith Momentum](https://arxiv.org/pdf/2007.07989.pdf): <br>
$ v_t = \gamma * v_ {t-1} + (1- \gamma) \nabla_ \theta f (\theta) $ <br>
$ \theta = \theta - \alpha v_t $ <br>
Where: <br> $ \nabla_ \theta f (\theta) $ - partial derivative with respect to $ \theta $ <br>
$ \gamma $ - acceleration weight. $ \gamma \in [0,1) $ <br>
$ \alpha $ - learning rate (shift step)

I will implement both and see where the MSE is less after the same number of steps

We take old class from [multivariant gradient descent](https://www.kaggle.com/konstantinsuspitsyn/multivariate-gradient-descent) and we keep progress_tracker & mse_function

In [5]:
class GradientDescents:
  

  import random

  def progress_tracker(self, step: int, cost_function: float) -> None:
    '''
    The function allows you to track online progress

    :param step: current step
    :param cost_function: the value of the cost function at the moment

    '''
    from IPython.display import clear_output
    clear_output(wait=True)
    print('Step: {}'.format(step))
    print('Loss: {:.2f}'.format(cost_function))

  def mse_function(self, y_true: list, y_pred: list) -> float:
    '''
    Function that calculates MSE

    :param y_true: the y values we know from the actual data
    :param y_pred: the y values we got at the moment

    :return mse: MSE value
    '''
    # Number of values to compare with actual y
    n = len(y_true)
    # Starting with 0
    pre_mse = 0
    for index, value in enumerate(y_true):
      pre_mse += (value - y_pred[index])**2
    mse = pre_mse/n
    return mse

  def gradient_descent_multi(self, X_true: list, y_true: list, \
                              weights: list = None, max_steps: int = 10000, \
                              learning_rate: float = 0.003, \
                              save_steps: int = 0) -> dict:
    '''
    Gradient descent for multiple variables

    :param X_true: actual attributes
    :param y_true: actual results
    :param weights: starting weights, if we don't want to start training from random
    :param learning_rate: learning rate
    :param max_steps: maximum number of steps at which the algorithm will stop
    :param save_steps: if 0, only last step will be saved
                       If not 0, every #save_steps will be saved
    
    :return {
      :return weights: regression weights
      :return mse: MSE
      :return steps: # of Steps
      :return mse_list: list of MSEs if save_steps > 0
      :return weights_list: list of weigtht lists if save_steps > 0
    }
    '''

    # Code for data with only one atribute
    if (type(X_true[0])==int) or (type(X_true[0])==float):
      for i, x in enumerate(X_true):
        X_true[i]=[x,1]
    elif (type(X_true[0])==list) and (len(X_true[0])==1):
      for i, x in enumerate(X_true):
        X_true[i].append(1)

    # Initialize random weights
    if weights == None:
      weights = [self.random.random() for f in X_true[0]]

    if save_steps > 0:
      mse_list = []
      weights_list = []
    
    # MSE of the previous state
    mse_prev = 0
    mse = 999

    # Nubmer of experiments we've got
    n = len(X_true)

    step = 0
    while (step <= max_steps) and (abs(mse_prev-mse)>1e-5):
      # Calculate gradients
      gradients = []
      for wi, w_value in enumerate(weights):
        current_gradient=0
        for yi, y_t_val in enumerate(y_true):
          current_gradient += -2*(y_t_val - sum([w*x for w,x in \
                                                 zip(weights,X_true[yi])]))* X_true[yi][wi]
        current_gradient = current_gradient/n
        gradients.append(current_gradient)

      # Change weights
      for gi, gr_value in enumerate(gradients):
        weights[gi] = weights[gi] - learning_rate*gr_value

      # Calculate y_pred
      y_pred = []
      for X_current in X_true:
        y_pred.append(sum([w*x for w,x in zip(weights,X_current)]))
      
      step +=1
      mse_prev = mse
      mse = self.mse_function(y_true, y_pred)
      self.progress_tracker(step, mse)

      if save_steps > 0:
        if step % save_steps == 0:
          mse_list.append(mse)
          weights_list.append(weights)

    if save_steps > 0:
      return_dict = {'weights': weights, 'mse':mse, 'steps': step-1, \
                      'mse_list': mse_list, 'weights_list': weights_list}
    else:
      return_dict = {'weights': weights, 'mse':mse, 'steps': step-1}

    return return_dict

In [6]:
new_grad = GradientDescents()

Realisation
1. [An overview of gradient descent optimization algorithms](https://arxiv.org/pdf/1609.04747.pdf):<br>
$v_t = \gamma * v_{t-1} + \alpha \nabla_\theta f(\theta)$<br>
$\theta = \theta - v_t$

In [4]:
import random

In [2]:
# Loading Boston Dataset
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)
X_true = []
for i in X:
  x_s_list = [f for f in i]
  # x_s_list.append(1)
  X_true.append(x_s_list)
y_true = [f for f in y]
del X, y

In [ ]:
learning_rate = 0.0000003
max_steps = 50000
step = 0

# Momentum
gamma = 0.5

# For the purity of the choice of the algorithm, I will start the weights from 1
weights = [1] * len(X_true[0])

# Number of elements in row of X
n = len(X_true[0])

# Gradients
# Current gradient
gradient = []
# Previous move
v_t_previous = [0] * len(X_true[0])

all_mses_algo1 = []

while step < max_steps:
  # For experiment we will equal seed with step
  random.seed(step)
  # Get random row
  index = random.randint(0, n-1)
  # X & y for current step
  X_current = X_true[index]
  y_current = y_true[index]
  gradient = []
  # Calculate current gradient
  for x_i in X_current:
    current_gradient = -2*(y_current - sum([w*x for w,x in \
                                          zip(weights,X_current)]))*x_i
    gradient.append(current_gradient)
  
  # Apply momentum for previous step
  momentum_v_t_previous = [f*gamma for f in v_t_previous]
  # Applying step for gradient
  step_gradient = [f*learning_rate for f in gradient]
  # New delta to move weights
  v_t = [a+b for a,b in zip(momentum_v_t_previous,step_gradient)]
  v_t_previous = v_t

  # Move weights
  for vti, vti_value in enumerate(v_t):
    weights[vti] = weights[vti] - vti_value

  y_pred = sum([w*x for w,x in zip(weights,X_current)])
  mse = new_grad.mse_function([y_pred], [y_current])

  step += 1

  new_grad.progress_tracker(step, mse)

  # Check on progress
  y_pred_algo_1 = []
  for X_current in X_true:
    y_pred_algo_1.append(sum([w*x for w,x in zip(weights,X_current)]))

  mse_algo_1 = new_grad.mse_function(y_pred_algo_1, y_true)


  all_mses_algo1.append(mse_algo_1)

Step: 3892
Loss: 120.08


Try second algorithm<br>
2. [An Improved Analysis of Stochastic Gradient Descentwith Momentum](https://arxiv.org/pdf/2007.07989.pdf):<br>
$v_t = \gamma * v_{t-1} + (1-\gamma) \nabla_\theta f(\theta)$<br>
$\theta = \theta - \alpha v_t$<br>

In [ ]:
learning_rate = 0.0000003
max_steps = 50000
step = 0

# Momentum
gamma = 0.9

# For the purity of the choice of the algorithm, I will start the weights from 1
weights = [1] * len(X_true[0])

# Number of elements in row of X
n = len(X_true[0])

# Gradients
# Current gradient
gradient = []
# Previous move
v_t_previous = [0] * len(X_true[0])
all_mses_algo2 = []
while step < max_steps:
  # For experiment we will equal seed with step
  random.seed(step)
  # Get random row
  index = random.randint(0, n-1)
  # X и y for current step
  X_current = X_true[index]
  y_current = y_true[index]
  gradient = []
  # Calculating current gradient
  for x_i in X_current:
    current_gradient = -2*(y_current - sum([w*x for w,x in \
                                          zip(weights,X_current)]))*x_i
    gradient.append(current_gradient)
  
  # Apply momentum for previous step
  momentum_v_t_previous = [f*gamma for f in v_t_previous]
  # Applying step for gradient
  step_gradient = [f*(1-gamma) for f in gradient]
  # New delta to move weights
  v_t = [a+b for a,b in zip(momentum_v_t_previous,step_gradient)]
  # В предыдущий сдвиг записываем новый
  v_t_previous = v_t

  # Make weights move
  for vti, vti_value in enumerate(v_t):
    weights[vti] = weights[vti] - vti_value * learning_rate

  y_pred = sum([w*x for w,x in zip(weights,X_current)])
  mse = new_grad.mse_function([y_pred], [y_current])

  step += 1

  new_grad.progress_tracker(step, mse)

  # Follow the progress
  y_pred_algo_2 = []
  for X_current in X_true:
    y_pred_algo_2.append(sum([w*x for w,x in zip(weights,X_current)]))

  mse_algo_2 = new_grad.mse_function(y_pred_algo_2, y_true)

  all_mses_algo2.append(mse_algo_2)


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
steps = [i+1 for i, f in enumerate(all_mses_algo2)]

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot()
ax.plot(steps, all_mses_algo1, color='#ff6361', \
        label='First model')
plt.plot(steps, all_mses_algo2, color='#ffa600', \
         label='Second model')
plt.title('Models Comparison All')
ax.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot()
ax.plot(steps[:500], all_mses_algo1[:500], color='#ff6361', \
        label='First model')
plt.plot(steps[:500], all_mses_algo2[:500], color='#ffa600', \
         label='Second model')
plt.title('Models Comparison Head')
ax.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot()
ax.plot(steps[-2000:], all_mses_algo1[-2000:], color='#ff6361', \
        label='First model')
plt.plot(steps[-2000:], all_mses_algo2[-2000:], color='#ffa600', \
         label='Second model')
plt.title('Models Comparison Tail')
ax.legend()
plt.show()

Tests show that the first model is more efficient. We will put it in the class

# Final Class

In [ ]:
class GradientDescents:
  
  '''
  Gradient descent from scratch
  '''

  import random

  def progress_tracker(self, step: int, cost_function: float) -> None:
    '''
    The function allows you to track online progress

    :param step: current step
    :param cost_function: the value of the cost function at the moment

    '''
    from IPython.display import clear_output
    clear_output(wait=True)
    print('Step: {}'.format(step))
    print('Loss: {:.2f}'.format(cost_function))

  def mse_function(self, y_true: list, y_pred: list) -> float:
    '''
    Function that calculates MSE

    :param y_true: the y values we know from the actual data
    :param y_pred: the y values we got at the moment

    :return mse: MSE value
    '''
    # Number of values ​​to compare
    n = len(y_true)
    # Starting with 0
    pre_mse = 0
    for index, value in enumerate(y_true):
      pre_mse += (value - y_pred[index])**2
    mse = pre_mse/n
    return mse

  def gradient_descent_multi(self, X_true: list, y_true: list, \
                              weights: list = None, max_steps: int = 10000, \
                              learning_rate: float = 0.003, \
                              save_steps: int = 0) -> dict:
    '''
    Gradient descent for multiple variables

    :param X_true: actual attributes
    :param y_true: actual results
    :param weights: starting weights, if we don't want to start training from random
    :param learning_rate: learning rate
    :param max_steps: maximum number of steps at which the algorithm will stop
    :param save_steps: if 0, only last step will be saved
                       If not 0, every #save_steps will be saved
    
    :return {
      :return weights: regression weights
      :return mse: MSE
      :return steps: # of Steps
      :return mse_list: list of MSEs if save_steps > 0
      :return weights_list: list of weigtht lists if save_steps > 0
    }
    '''

    # Code for data with one attribute
    if (type(X_true[0])==int) or (type(X_true[0])==float):
      for i, x in enumerate(X_true):
        X_true[i]=[x,1]
    elif (type(X_true[0])==list) and (len(X_true[0])==1):
      for i, x in enumerate(X_true):
        X_true[i].append(1)

    # Initizlize random weights
    if weights == None:
      weights = [self.random.random() for f in X_true[0]]

    if save_steps > 0:
      mse_list = []
      weights_list = []
    
    # previous step MSE
    mse_prev = 0
    mse = 999

    # Number of experiments we got     
    n = len(X_true)

    step = 0
    while (step <= max_steps) and (abs(mse_prev-mse)>1e-5):
      # Count gratients
      gradients = []
      for wi, w_value in enumerate(weights):
        current_gradient=0
        for yi, y_t_val in enumerate(y_true):
          current_gradient += -2*(y_t_val - sum([w*x for w,x in \
                                                 zip(weights,X_true[yi])]))* X_true[yi][wi]
        current_gradient = current_gradient/n
        gradients.append(current_gradient)

      # Move weights
      for gi, gr_value in enumerate(gradients):
        weights[gi] = weights[gi] - learning_rate*gr_value

      # Calculate y_pred
      y_pred = []
      for X_current in X_true:
        y_pred.append(sum([w*x for w,x in zip(weights,X_current)]))
      
      step +=1
      mse_prev = mse
      mse = self.mse_function(y_true, y_pred)
      self.progress_tracker(step, mse)

      if save_steps > 0:
        if step % save_steps == 0:
          mse_list.append(mse)
          weights_list.append(weights)

    if save_steps > 0:
      return_dict = {'weights': weights, 'mse':mse, 'steps': step-1, \
                      'mse_list': mse_list, 'weights_list': weights_list}
    else:
      return_dict = {'weights': weights, 'mse':mse, 'steps': step-1}

    return return_dict

  def momentum_gradient_descent(self, X_true: list, y_true: list, \
                                weights: list = None, max_steps: int = 10000, \
                                learning_rate: float = 0.003, gamma: float = 0.9, \
                                save_steps: int = 0) -> dict:
    '''
    Gradient descent with acceleration

    : param X_true: actual attributes
    : param y_true: actual results
    : param weights: starting weights, if we don't want to start training from random
    : param learning_rate: learning rate
    : param max_steps: maximum number of steps at which the algorithm will stop
    : param save_steps: if 0, only the last step will be saved
                        if the value is nonzero,
                        every i-th step will be saved


    : return {
      : return weights: regression weights
      : return mse: MSE value
      : return steps: number of steps
      : return mse_list: MSE value during training if save_steps> 0
      : return weights_list: weights as we learn if save_steps> 0
    }
                      
    '''
    # Code for data with one attribute
    if (type(X_true[0])==int) or (type(X_true[0])==float):
      for i, x in enumerate(X_true):
        X_true[i]=[x,1]
    elif (type(X_true[0])==list) and (len(X_true[0])==1):
      for i, x in enumerate(X_true):
        X_true[i].append(1)


    if save_steps > 0:
      mse_list = []
      weights_list = []

    step = 0
    mse_prev = 999999999


    # Random weights
    if weights == None:
      weights = [self.random.random() for f in X_true[0]]

    # Amount of elemets in row X
    n = len(X_true[0])

    # Gradients
    # Current gradient
    gradient = []
    # Previous move
    v_t_previous = [0] * len(X_true[0])
    
    # TO DO: Change it to tain-validation datasets
    while step < max_steps:

      # We take a random number to select data
      index = self.random.randint(0, n-1)
      # X & y for current step
      X_current = X_true[index]
      y_current = y_true[index]
      gradient = []
      # Calculate current gradient
      for x_i in X_current:
        current_gradient = -2*(y_current - sum([w*x for w,x in \
                                              zip(weights,X_current)]))*x_i
        gradient.append(current_gradient)
      
      # Apply momentum
      momentum_v_t_previous = [f*gamma for f in v_t_previous]
      # Apply step to a gradient
      step_gradient = [f*learning_rate for f in gradient]
      # Get a new move
      v_t = [a+b for a,b in zip(momentum_v_t_previous,step_gradient)]
      v_t_previous = v_t

      # Make a move to weights
      for vti, vti_value in enumerate(v_t):
        weights[vti] = weights[vti] - vti_value

      y_pred = sum([w*x for w,x in zip(weights,X_current)])

      y_pred_algo = []
      for X_current in X_true:
        y_pred_algo.append(sum([w*x for w,x in zip(weights,X_current)]))

      
      mse = self.mse_function(y_pred_algo, y_true)

      if mse < mse_prev:
        # Saving only best results
        final_weights = weights

      mse_prev = mse

      step += 1

      self.progress_tracker(step, mse)

      if save_steps > 0:
        if step % save_steps == 0:
          mse_list.append(mse)
          weights_list.append(weights)

    if save_steps > 0:
      return_dict = {'weights': final_weights, 'mse':mse, 'steps': step-1, \
                      'mse_list': mse_list, 'weights_list': weights_list}
    else:
      return_dict = {'weights': final_weights, 'mse':mse, 'steps': step-1}

    return return_dict

# Test on Boston Dataset

In [ ]:
# Load Boston Dataset
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)
X_true = []
for i in X:
  x_s_list = [f for f in i]
  # x_s_list.append(1)
  X_true.append(x_s_list)
y_true = [f for f in y]
del X, y

In [ ]:
msgd = GradientDescents()

In [ ]:
gd = msgd.momentum_gradient_descent(X_true, y_true, learning_rate=0.000003, max_steps=1000)

In [ ]:
y_pred_algo_1 = []
for X_current in X_true:
  y_pred_algo_1.append(sum([w*x for w,x in zip(gd['weights'],X_current)]))

mse_algo_1 = msgd.mse_function(y_pred_algo_1, y_true)

In [ ]:
gd_next = msgd.momentum_gradient_descent(X_true, y_true, learning_rate=0.000003, max_steps=10000,\
                                         weights = gd['weights'])

In [ ]:
y_pred_algo_2 = []
for X_current in X_true:
  y_pred_algo_2.append(sum([w*x for w,x in zip(gd_next['weights'],X_current)]))

mse_algo_2 = msgd.mse_function(y_pred_algo_2, y_true)

In [ ]:
mse_algo_1

In [ ]:
mse_algo_2

In [ ]:
gd_next = msgd.momentum_gradient_descent(X_true, y_true, learning_rate=0.000003, max_steps=50000,\
                                         weights = gd_next['weights'])

In [ ]:
y_pred_algo_3 = []
for X_current in X_true:
  y_pred_algo_3.append(sum([w*x for w,x in zip(gd_next['weights'],X_current)]))

mse_algo_3 = msgd.mse_function(y_pred_algo_3, y_true)

In [ ]:
mse_algo_3

Of course, it would be more correct to calculate MSE on a test dataset, but we see that the result improves as we train